# 系列データのモデル化-リカレントニューラルネットワーク

本章ではリカレントニューラルネットワーク(RNN:Recurrent Neural Network)に焦点を合わせ，系列データのモデル化への応用と，系列データの一種である時系列データを取り上げる．本章では，次の項目を取り上げる．

- 系列データの概要
- RNN：シーケンスモデルの構築
- **長短期記憶**(LSTM)
- **T-BPTT**(Truncated Backpropagation Through Time)
- シーケンスモデルを構築するためのTensorFlowでの多層RNNの実装
- プロジェクト1：RNNによるIMDb映画レビューデータセットの感情分析
- プロジェクト2：シェイクスピアの『ハムレット』を使って文字レベルの言語モデルをRNNモデルとして構築
- 勾配の発散を回避するための勾配刈り込みの使用

## 系列データ

RNNを説明するにあたってまずは系列データの性質から見ていく．

より一般的には系列データは**シーケンス**(sequence)と呼ばれる．
ここでは，系列データのユニークな特性を調べることで，他の種類のデータとどのように異なるのかを明らかにする.

### 系列データのモデル化

系列データ(シーケンス)と他の種類のデータとの違いは，シーケンスでは要素が特定の順序で並んでいて，互いに無関係ではないことにある．

教師あり機械学習の典型的な機械学習アルゴリズムでは，入力データが**独立同分布**(Independent and Identically Distributed:IID)であることが前提となる．

例えば，$n$個のデータサンプル$x^{(1)},x^{(2)},\cdots,x^{(n)}$がある場合，このデータを機械学習のトレーニングに使用するときの順序は問題ではない．

しかし，シーケンスを扱うときには，この前提は成り立たなくなる．当然ながら，順序は重要だからである．

### 系列データを表現する

入力データでは，シーケンスの要素が互いに依存する順序で並んでいることが裏付けられたとする．

本章では，シーケンスを$(x^{(1)},x^{(2)},\cdots,x^{(T)})$で表すことにする．上付き文字はインスタンスの順序を表しており，シーケンスの長さは$T$である．

さて，シーケンスといえば，時系列データである．時系列データでは，各サンプル点$x^{(t)}$が特定の時間$t$に属している．

多層パーセプトロン(MLP)や畳み込みニューラルネットワーク(CNN)と言った標準のニューラルネットワークモデルは，入力サンプルの**順序**を処理できない．

直感的に言えるのは，そうしたモデルが過去に検出したサンプルの**記憶**を持たない事である．例えば，それらのサンプルはフィードフォワードステップとバックプロパゲーションステップを通過していく．そして，それらの重みは，サンプルが処理される順序とは無関係に更新される．

対照的に，リカレントニューラルネットワーク(RNN)の目的は，シーケンスを設計し，モデル化することにある．RNNは過去の情報を記憶しておき，その情報に従って新しい事象を処理できる．

### シーケンスモデルの様々なカテゴリ

シーケンスモデルには，魅力的な応用例がいくつもある．例えば，言語翻訳，画像キャプショニング，テキスト生成などが挙げられる．

ただし，適切なモデルを開発するには，シーケンスモデルを構築するための様々な種類のタスクを理解する必要がある．

入力データと出力データのどちらかがシーケンスであるとしたら，そのデータは次の3種類のカテゴリのいずれかに分類される．

- **多対一**
  - 入力データはシーケンスだが，出力は(シーケンスではなく)固定サイズのベクトルである．例えば感情分析では，入力はテキストベースであり，出力はクラスラベルである．
- **一対多**
 - 入力データは標準フォーマットであり，シーケンスではないが，出力はシーケンスである．このカテゴリの一例は画像キャプショニングである．画像キャプショニングでは，入力は画像であり，出力は英語のフレーズである．
- **多対多**
 - 入力データと出力データはどちらもシーケンスである．このカテゴリは，入力と出力が同期するかどうかに基づいて更に分類できる．多対多の**同期モデル**の一例は動画分類である．動画分類では，動画の各フレームがラベル付けされる．多対多の**遅延**モデルの一例は言語翻訳である．例えば，英語をドイツ語に翻訳するときには，英語の文章全体を読み込んで処理したうえでドイツ語に翻訳しなければならない．

シーケンスモデルのカテゴリを理解したところで，RNNの構造について見ていく．

## リカレントニューラルネットワーク:シーケンスモデルの構築

### RNNの構造とデータの流れ

標準のフィードフォワードニューラルネットワークでは，情報は入力層から隠れ層へ流れ，隠れ層から出力層へ流れる．これに対し，RNNでは，隠れ層の入力は入力層から得られるだけでなく，**1つ前の時間刻みの隠れ層からも得られる**．

隠れ層の連続する時間刻みの間を情報が流れることにより，ネットワークが過去の事象に対する記憶を持つことが可能になる．こうした情報の流れは，通常はループ(循環)として表示される．グラフ表記ではこのループは**リカレントエッジ**(recurrent edge)とも呼ばれる．このアーキテクチャ全体を「リカレントニューラルネットワーク」と呼ぶのはそのためである．

知っての通り，標準的なニューラルネットワークの隠れユニットが受け取る入力はそれぞれ1つだけである．隠れユニットの入力は，入力層に関連付けられた事前活性化(総入力)である．対照的に，RNNの隠れユニットはそれぞれ，入力層からの事前活性化と，1つ前の時間刻み$t-1$の同じ隠れ層からの活性化という2つの入力を受け取る．

最初の時間刻み$t=0$では，隠れユニットはそれぞれ0または小さな乱数で初期化される．次に，$t\gt0$の時間刻みでは，隠れユニットは2つの場所から入力を受けとる．それらは，現在の時間のデータ点$x^{(t)}$と，1つ前の時間刻み$t-1$の隠れユニットの値$h^{(t-1)}$である．

同様に多層RNNの場合は情報の流れを次のようにまとめることが出来る．

- **layer=1**
 - この場合，隠れ層は$h_{1}^{(t)}$で表される．隠れ層の入力は，データ点$x^{(t)}$と，同じ層の1つ前の時間刻みの隠れユニットの値$h_{1}^{(t-1)}$である．
- **layer=2**
 - 2つ目の隠れ層$h_{2}^{(t)}$は，現在の時間刻みの下にある隠れユニット$h_{1}^{(t)}$と，同じ層の1つ前の時間刻みの隠れユニット$h_{2}^{(t-1)}$から入力を受け取る．

### 長期的な相互作用の学習と勾配消失・発散問題

RNNのトレーニング手法の1つにBPTT(Backpropagation Through Time)がある．これは，損失関数の勾配を計算するときの乗法係数$\frac{\partial h^{(t)}}{\partial h^{(k)}}$により，いわゆる**勾配消失**(vanishing gradient)問題と**勾配発散**(exploding gradient)問題が発生する．

直感的に分かるのは，勾配消失問題や，勾配発散問題を回避するための単純な解決策が，$|w|=1$を保証することによって実現できることである．

実際にはこの問題に対する解決策が2つある．

- T-BPTT(Truncated Backpropagation Through Time)
- 長短期記憶(Long Short-Term Memory:LSTM)

T-BPTTは指定されたしきい値の上で勾配を刈り込む．T-BPTTは勾配発散問題を解決できるが，この刈り込みにより，勾配とは逆方向に進んで重みを正しく更新できる時間刻みの数が制限される．

一方で，LSTMは勾配消失問題を克服することにより，シーケンスでの長期的な依存関係のモデル化において成功を収めている．

以下でLSTMについて少し詳しく見ていく．

### LSTMのユニット

LSTMの構成要素は**メモリセル**(memory cell)である．メモリセルは基本的には隠れ層を表す．

各メモリセルには，勾配消失問題と勾配発散問題を克服するためのリカレントエッジが存在する．

先に述べたように，リカレントエッジの望ましい重みは$w=1$である．このリカレントエッジに関連付けられる値は**セル状態**(cell state)と呼ばれる．次の図は，LSTMセルの構造を展開したものである．

![LSTMセル](images/lstm_cell.jpg)

この図から分かるように，1つ前のセル状態$C^{(t-1)}$が，現在の時間刻みのセル状態を取得するために(重み係数を直接かけることなく)変更される．

このメモリセルでの乗法の流れは，次に説明する計算ユニットによって制御される．

4つのボックスには，活性化関数(シグモイド関数$\sigma$, 双曲線正接関数tanh)といちれんの重みが含まれている．
これらのボックスでは入力で行列とベクトルの乗算を行うことで，線型結合を適用する．

これらの計算つニットとシグモイド活性化関数は**ゲート**(gate)と呼ばれる．ゲートの出力ユニットは$\odot$を通じて渡される．

LSTMセルには，忘却ゲート，入力ゲート，出力ゲートの3種類のゲートが存在する．

- **忘却ゲート**($f_{t}$)では，メモリセルを無限に成長させるのではなく，セル状態をリセットできる．実際には，忘却ゲートは通過させる情報と通過させない情報を決定する．$f_{t}$は次の様に計算される． $$f_{t}=\sigma(W_{xf}x^{(t)}+W_{hf}h^{(t-1)}+b_{f})$$
なお，忘却ゲートは最初からLSTMのセルの一部だったわけではなく，最初のモデルを改善するために数年後に追加されたものである．

- **入力ゲート**($i_{t}$)と入力ノード($g_{t}$)は，セル状態を更新する役割を果たす．入力ゲートと入力ノードは次のように計算される．
$$i_{t}=\sigma(W_{xi}x^{(t)}+W_{hi}h^{(t-1)}+b_{i}) \\ g_{t}=\tanh(W_{xg}x^{(t)}+W_{hg}h^{(t-1)}+b_{g})$$
時間$t$でのセル状態は次のように計算される． $$C^{(t)}=(C^{(t-1)}\odot f_{t})\oplus (i_{t}\odot g_{t})$$

- **出力ゲート**($o_{t}$)は，隠れユニットの値の更新方法を決定する．
$$o_{t}=\sigma(W_{xo}x^{(t)}+W_{ho}h^{(t-1)}+b_{o})$$
したがって，現在の時間刻みでの隠れユニットは次のように計算される．
$$h^{(t)}=o_{t}\odot \tanh(C^{(t)})$$

LSTMセルの構造とそのベースとなる計算は，かなり複雑に思えるかもしれないが，TensorFlowにはLSTMセルを簡単に定義できるラッパー関数が一通り実装されている．

## 多層RNNの実装

### プロジェクト1:IMDb(Internet Movie Database)映画レビューの感情分析

感情分析では，文章またはテキスト文書に表明されている意見を分析する．

ここでは多対一のアーキテクチャに基づいて感情分析をするための多層RNNを実装する．

#### データの準備

映画レビューデータセットはgzipで圧縮されたアーカイブとして[ここ](https://ai.stanford.edu/~amaas/data/sentiment/)で配布されているのでダウンロードして展開しておく．

そして，次に示すコードを用いてダウンロードアーカイブに含まれていたテキスト文書を1つのCSVファイルにまとめる．

In [5]:
import pyprind
import pandas as pd
import numpy as np
import os

from __future__ import print_function, division

In [8]:
basepath = '/home/yokota/datasets/aclImdb'
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()



0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:00


In [18]:
# 行の順番をシャッフル
df = df.sample(frac=True, random_state=0).reset_index(drop=True)

# CSVに変換
df.to_csv('/home/yokota/datasets/aclImdb/move_data.csv', index=False, encoding='utf-8')

csv_filepath = '/home/yokota/datasets/aclImdb/move_data.csv'
df = pd.read_csv(csv_filepath, encoding='utf-8')
df.head(3)

,0,1
0,"The cast of ""All That"" returns for good humor ...",0
1,"Considering it was made on a low budget, THE D...",1
2,Luise Rainer received an Oscar for her perform...,0


このdfデータフレームには映画レビューのテキストが含まれる'review'列('0')と0または1のラベルが含まれる'sentiment'('1')の2つの列が含まれている．

これらの映画レビューのテキストコンポーネントは，単語のシーケンスである．このため，各シーケンスの単語を処理するRNNモデルを構築し，最終的には文章全体を0または1のクラスに分類したい．

ニューラルネットワークへの入力データを準備するには，このデータを数値としてエンコードする必要がある．

そのためにはまず，データセット全体から一意な単語を見つけ出す．これにはPythonのset(集合)を利用できるが，こうした大きなデータセットから一意な単語を見つけ出すことはあまり効率的ではないことが分かった．

それよりも効率的なのは，collectionsパッケージのCounterを使用する方法である．

次のコードではCounterクラスからcountsオブジェクトを作成する．このクラスは，テキストに含まれている一意な単語ごとに出現回数をカウントする．BoW(Bag-of-Words)モデルとは対照的に，このアプリケーションの関心は一意な単語の集まりだけであることに注意する(単語の出現回数には関心がない)．

次にマッピングを作成する．このマッピングは，このデータセットの一意な単語をそれぞれ一意な整数にマッピングするディクショナリとして作成する．このディクショナリの名前はword_to_intであり，映画レビューのテキスト全体を数値のリストに変換するために使用できる．検出された一意な単語は出現回数を元にソートされるが，任意の順序を使用したとしても最終的な結果への影響はない．

この「テキストを一連の整数に変換する」プロセスのコードは次のようになる．

In [25]:
from string import punctuation
import re
from collections import Counter

In [26]:
# データの前処理：単語を分割し，各単語の出現回数をカウント
counts = Counter()
pbar = pyprind.ProgBar(len(df['0']), title='Counting words occurrences')

for i, review in enumerate(df['0']):
    text = ''.join([c if c not in punctuation else ' ' + c + ' '
                    for c in review]).lower()
    df.loc[i, '0'] = text
    pbar.update()
    counts.update(text.split())

# マッピングを作成：一意な単語をそれぞれ整数にマッピング
word_counts = sorted(counts, key=counts.get, reverse=True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in enumerate(word_counts, 1)}

mapped_reviews = []
pbar = pyprind.ProgBar(len(df['0']), title='Map reviews to ints')
for review in df['0']:
    mapped_reviews.append([word_to_int[word] for word in review.split()])
    pbar.update()

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:55
Map reviews to ints


[u'the', u'.', u',', u'and', u'a']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


これで，単語のシーケンスが整数のシーケンスに変換された．ただし，解決しなければならない課題が1つ残っている．

現時点では，シーケンスの長さはまちまちである．このRNNアーキテクチャと互換性がある入力データを生成するには全てのシーケンスを同じ長さにする必要がある．

そこで，sequence_lengthというパラメータを定義し，200に設定する．シーケンスの長さが200語未満の場合は，シーケンスの左側を0でパディングする．逆に200語を超える場合には，最後の200語だけが使用されるように切り取る．

この前処理ステップは，次の2つの手順で実装できる．

1. 要素が0の行列を作成する．この行列の各行は，サイズが200のシーケンスに対応する．
1. 各シーケンスの単語のインデックスを行列の右側から埋めていく．

sequence_lengthは実際にはハイパーパラメータであり，パフォーマンスを最適化するためにチューニングを行うことが出来る．

これらの手順を実装して同じ長さのシーケンスを作成するコードは次のようになる．

In [37]:
sequence_length = 200 # シーケンスの長さ(RNNの式中のT)
sequence = np.zeros((len(mapped_reviews), sequence_length), dtype=int)

for i, row in enumerate(mapped_reviews):
    review_arr = np.array(row)
    sequence[i, -len(row):] = review_arr[-sequence_length:]

データセットの前処理を行った後は，トレーニングデータセットとテストデータセットの分割を行う．

このデータセットはすでにシャッフル済みであるため，データセットの前半分をトレーニングに使用し，後半分をテストに使用すれば良い．

In [38]:
X_train = sequence[:25000, :]
y_train = df.loc[:25000, '1'].values
X_test = sequence[25000:, :]
y_test = df.loc[25000:, '1'].values

データセットを交差検証のために分割したい場合は，データセットの後半分を更に分割することで，一回り小さなテストセットと，ハイパーパラメータを最適化するための検証セットを作成することが出来る．

最後に，ヘルパー関数を定義する．この関数は，与えられたデータセット(トレーニングデータセット，テストデータセットの場合がある)をチャンクに分割し，これらのチャンクを反復的に処理するためのジェネレータを返す．こうしたチャンクは**ミニバッチ**(mini-batch)とも呼ばれる．

In [39]:
np.random.seed(123)

# ミニバッチを生成する関数を定義
def create_batch_generator(x, y=None, batch_size=64):
    n_batches = len(x) // batch_size
    x = x[:n_batches * batch_size]
    if y is not None:
        y = y[:n_batches * batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        else:
            yield x[ii:ii+batch_size]

こうしたジェネレータを使用する方法は，メモリの制限に対処するのに非常に効果的である．

ニューラルネットワークのトレーニングでは，全てのデータを事前に分割してトレーニングが完了するまでメモリ内で保持するのではなく，データセットをミニバッチに分割する方法が推奨される．

#### 埋め込み

前項では，データの前処理で同じ長さのシーケンスを生成した．これらのシーケンスの要素は，一意な単語の**インデックス**に対応する整数だった．

こうした単語のインデックスを入力特徴量に変換する方法は何種類かある．単純な方法の1つはone-hotエンコーディングを適用することでインデックスを0と1のベクトルに変換することである．

しかし，入力データ数が大きくなると入力特徴量が大きくなり過ぎでモデルが**次元の呪い**に陥るかもしれない．さらに，これらの特徴量は1つを除いて0であるためかなり疎な特徴量である．

より洗練された方法は，実数値の(整数であるとは限らない)要素を持つ固定サイズのベクトルに各単語をマッピングすることである．one-hotエンコーディングのベクトルとは対照的に，有限サイズのベクトルを使って無数の実数を表すことが出来る．理論的には$[-1, 1]$などの区間から実数を無限に抽出できる．

これがいわゆる**埋め込み**(embedding)の考え方である．埋め込みは表現学習(目的に適した特徴量ベクトルを学習を通して得る方法，得られる特徴量ベクトルは**分散表現**と呼ばれる)の手法の1つであり，ここでは，データセットの単語を表す顕著な特徴量を自動的に学習するために利用できる．

一意な単語の個数がunique\_words，埋め込みベクトルのサイズがembedding_sizeであるとする．語彙全体を入力特徴量として表すには，embedding_sizeの値をunique\_wordsよりもかなり小さくすれば良い(embedding\_size$\ll$unique\_words)．

埋め込みには，one-hotエンコーディングよりも有利な点が2つある．

- 次元の呪いの影響を抑制する特徴空間の次元削減
- ニューラルネットワークの埋め込み層がトレーニング可能であることによる顕著な特徴量の抽出

TensorFlowにはtf.nn.embedding\_lookupという効率的な関数が実装されている．この関数は，一意な単語に対応する各整数を，トレーニング可能な行列の行にマッピングする．

次に，埋め込み層を実際に作成する方法を見てみる．入力層がtf_xで，対応する語彙のインデックスがtf.int32型で供給されるとすれば，次の2つの手順に従って埋め込み層を作成することが出来る．

1. まず，サイズがn\_words$\times$embedding\_sizeの行列をembeddingというテンソル変数として作成する．そして，この行列の要素を$[-1,1]$の浮動小数点型の乱数で初期化する．
1. 次に，tf.nn.embedding\_lookup関数を呼び出し，tf_xの各要素に関連する埋め込み行列の行を特定する．

In [40]:
import tensorflow as tf

/home/yokota/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [48]:
g = tf.Graph()

with g.as_default():
    tf_x = tf.placeholder(tf.int32, shape=[], name='tf_x')
    n_words = len(word_to_int)
    embedding_size = 100
    
    embedding = tf.Variable(tf.random_uniform(shape=(n_words, embedding_size),
                            minval=-1, maxval=1))
    embed_x = tf.nn.embedding_lookup(embedding, tf_x)

#### RNNモデルの構築

RNNモデルを構築する準備が整ったところで，SentimentRNNクラスを実装してみよう．このクラスには次の4つのメソッドがある．

- **コンストラクタ**
 - モデルのパラメータを全て設定した後，計算グラフを作成し，self.buildメソッドを呼び出して多層RNNモデルを構築する．
- **buildメソッド**
 - 入力データ，入力ラベル，そして隠れ層のドロップアウト設定のキープ率に対応する3つのプレースホルダを宣言する．これらのプレースホルダを宣言した後，埋め込み層を作成し，埋め込み表現を入力として多層RNNを構築する．
- **trainメソッド**
 - 計算グラフを起動するためのTensorFlowセッションを作成し，計算グラフで定義されたコスト関数を最小化するために，ミニバッチを順番に処理しながら，指定された数のエポック数でトレーニングを行う．またチェックポイントとして10エポック後のモデルを保存する．
- **predictメソッド**
 - 新しいセッションを作成し，トレーニングプロセスで保存しておいた最後のチェックポイントを復元し，テストデータで予測値を生成する．
 
 このクラスとそのメソッドの実装を次に示す．

In [62]:
class SentimentRNN(object):
    def __init__(self, n_words, seq_len=200,
                 lstm_size=256, num_layers=1, batch_size=64,
                 learning_rate=0.0001, embed_size=200):
        """
        n_wordsの値は一意な単語の個数に等しくなければならない
        (トレーニングの際に1を足すのは長さ200未満のシーケンスを0パディングするためである)
        n_wordsは埋め込み層の作成時にembed_sizeハイパーパラメータと共に使用される．
        これに対し，seq_len変数は，先の前処理ステップで作成されたシーケンスの長さにしたがって設定されなければならない．
        lstm_sizeは，ここで使用しているハイパーパラメータの1つであり，RNNの各層の隠れユニットの個数を決定する．
        """
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size # 隠れユニットの個数
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
        
    def build(self):
        """
        buildメソッドではまずtf_x,tf_y,tf_keepprobの3つのプレースホルダを作成している．
        これらのプレースホルダは入力データを供給するために必要となる．
        次に埋め込み層を追加している．この埋め込み層により埋め込み表現embed_xが作成される．
        次にRNNモデルとLSTMセルを構築している．この部分は次の3つの手順に分かれている．
        1. 多層RNNモデルのセルを定義する．
        2. それらのセルの初期状態を定義する．
        3. セルとそれらの初期状態に基づいてRNNモデルを作成する．
        """
        tf_x = tf.placeholder(tf.int32, shape=(self.batch_size, self.seq_len), name='tf_x')
        tf_y = tf.placeholder(tf.float32, shape=(self.batch_size), name='tf_y')
        tf_keepprob = tf.placeholder(tf.float32, name='tf_keepprob')
        
        # 埋め込み層を作成
        embedding = tf.Variable(tf.random_uniform((self.n_words, self.embed_size),
                                                  minval=-1, maxval=1), name='embedding')
        embed_x = tf.nn.embedding_lookup(embedding, tf_x, name='embeded_x')
        
        # LSTMセルを定義し，組み上げる
        cells = tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(self.lstm_size),
                                           output_keep_prob=tf_keepprob)
            for i in range(self.num_layers)])
        
        # 初期状態を定義
        self.initial_state = cells.zero_state(self.batch_size, tf.float32)
        print('  << initial state >>  ', self.initial_state)
        
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, embed_x, initial_state=self.initial_state)
        # 注意：lstm_outputの形状：[batch_size, max_time, cells.output_size]
        print('\n  << lstm_output >>  ', lstm_outputs)
        print('\n  << final state >>  ', self.final_state)
        
        # RNNの出力の後に全結合層を適用
        logits = tf.layers.dense(inputs=lstm_outputs[:, -1],
                                 units=1, activation=None,
                                 name='logits')
        logits = tf.squeeze(logits, name='logits_squeezed')
        print('\n  << logits      >>  ', logits)
        
        y_proba = tf.nn.sigmoid(logits, name='probabilities')
        predictions = {
            'probabilities':y_proba,
            'labels':tf.cast(tf.round(y_proba), tf.int32, name='labels')
        }
        print('\n  << prediction  >>  ', predictions)
        
        # コスト関数を定義
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y,
                                                                      logits=logits,
                                                                      name='cost'))
        # オプティマイザを定義
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name='train_op')
        
    def train(self, X_train, y_train, num_epochs):
        """
        このtrainメソッドの実装では，各エポックの最初に，セルの現在の状態を初期状態にリセットしている．
        各ミニバッチの実行では，現在の状態に加えて，データbatch_xとそれらのラベルbatch_yを供給している．
        そしてミニバッチの実行の最後に，stateを最終状態に更新している．
        最終状態はtf.nn.dynamic_rnn関数から返される．この更新された状態は，次のミニバッチの実行に使用される．
        このプロセスが繰り返され，エポックを通じて現在の状態が更新される．
        """
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration = 1
            for epoch in range(num_epochs):
                state = sess.run(self.initial_state)
                
                for batch_x, batch_y in create_batch_generator(X_train, y_train, self.batch_size):
                    feed = {'tf_x:0':batch_x,
                            'tf_y:0':batch_y,
                            'tf_keepprob:0':0.5,
                            self.initial_state: state}
                    loss, _, state = sess.run(['cost:0', 'train_op', self.final_state],
                                              feed_dict=feed)
                    
                    if iteration % 20 == 0:
                        print('Epoch:{}/{} Iteration:{} | Train loss:{:.5f}'.format(
                               epoch + 1, num_epochs, iteration, loss.mean()))
                    iteration += 1
                
                if (epoch + 1) % 10 == 0:
                    self.saver.save(sess, 'models/sentiment-{}.ckpt'.format(epoch))
    
    def predict(self, X_data, return_proba=False):
        preds = []
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess, tf.train.latest_checkpoint('models/'))
            test_state = sess.run(self.initial_state)
            for ii, batch_x in enumerate(create_batch_generator(X_data, None, batch_size=self.batch_size), 1):
                feed = {'tf_x:0':batch_x,
                        'tf_keepprob:0':1.0,
                        self.initial_state:test_state}
            
            if return_proba:
                pred, test_state = sess.run(['probabilities:0', self.final_state], feed_dict=feed)
            else:
                pred, test_state = sess.run(['labels:0', self.final_state], feed_dict=feed)
            
            preds.append(pred)
        
        return np.concatenate(preds)
        

#### SentimentRNNクラスのインスタンス化

SentimentRNNクラスをインスタンス化するには，パラメータを次のように設定する．

In [64]:
n_words = max(list(word_to_int.values())) + 1

rnn = SentimentRNN(n_words=n_words,
                   seq_len=sequence_length,
                   embed_size=256,
                   lstm_size=128,
                   num_layers=1,
                   batch_size=100,
                   learning_rate=0.001)

  << initial state >>   (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(100, 128) dtype=float32>),)

  << lstm_output >>   Tensor("rnn/transpose_1:0", shape=(100, 200, 128), dtype=float32)

  << final state >>   (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(100, 128) dtype=float32>),)

  << logits      >>   Tensor("logits_squeezed:0", shape=(100,), dtype=float32)

  << prediction  >>   {'probabilities': <tf.Tensor 'probabilities:0' shape=(100,) dtype=float32>, 'labels': <tf.Tensor 'labels:0' shape=(100,) dtype=int32>}


ここでは単層RNNを使用するためにnum_layer=1を指定していることが分かる．ただし，この実装ではnum_layerに1よりも大きな値を設定すれば，多層RNNの作成が可能である．

ここで検討するデータセットは小さいため，トレーニングデータが過学習する可能性が低い単層RNNのほうが，未知のデータにうまく汎化することが考えられる．

#### 感情分析RNNモデルのトレーニングと最適化

次に，rnn.trainメソッドを呼び出すことで，このRNNモデルのトレーニングを行うことができる．このRNNモデルで40エポックのトレーニングを行うコードは次のようになる．

このトレーニングでは，X_trainに格納された入力データと，y_trainに格納された対応するクラスラベルを使用する．

In [60]:
rnn.train(X_train, y_train, num_epochs=40)

Epoch:1/40 Iteration:20 | Train loss:0.66575
Epoch:1/40 Iteration:40 | Train loss:0.64465
Epoch:1/40 Iteration:60 | Train loss:0.66747
Epoch:1/40 Iteration:80 | Train loss:0.61411
Epoch:1/40 Iteration:100 | Train loss:0.62158
Epoch:1/40 Iteration:120 | Train loss:0.50695
Epoch:1/40 Iteration:140 | Train loss:0.52379
Epoch:1/40 Iteration:160 | Train loss:0.51526
Epoch:1/40 Iteration:180 | Train loss:0.52912
Epoch:1/40 Iteration:200 | Train loss:0.52265
Epoch:1/40 Iteration:220 | Train loss:0.49093
Epoch:1/40 Iteration:240 | Train loss:0.48851
Epoch:2/40 Iteration:260 | Train loss:0.51594
Epoch:2/40 Iteration:280 | Train loss:0.40090
Epoch:2/40 Iteration:300 | Train loss:0.28087
Epoch:2/40 Iteration:320 | Train loss:0.46239
Epoch:2/40 Iteration:340 | Train loss:0.47271
Epoch:2/40 Iteration:360 | Train loss:0.29402
Epoch:2/40 Iteration:380 | Train loss:0.34497
Epoch:2/40 Iteration:400 | Train loss:0.31640
Epoch:2/40 Iteration:420 | Train loss:0.27542
Epoch:2/40 Iteration:440 | Train loss:

Epoch:15/40 Iteration:3520 | Train loss:0.00849
Epoch:15/40 Iteration:3540 | Train loss:0.04937
Epoch:15/40 Iteration:3560 | Train loss:0.04561
Epoch:15/40 Iteration:3580 | Train loss:0.00219
Epoch:15/40 Iteration:3600 | Train loss:0.00670
Epoch:15/40 Iteration:3620 | Train loss:0.00847
Epoch:15/40 Iteration:3640 | Train loss:0.00857
Epoch:15/40 Iteration:3660 | Train loss:0.00133
Epoch:15/40 Iteration:3680 | Train loss:0.00694
Epoch:15/40 Iteration:3700 | Train loss:0.02613
Epoch:15/40 Iteration:3720 | Train loss:0.00454
Epoch:15/40 Iteration:3740 | Train loss:0.02586
Epoch:16/40 Iteration:3760 | Train loss:0.02627
Epoch:16/40 Iteration:3780 | Train loss:0.00119
Epoch:16/40 Iteration:3800 | Train loss:0.01931
Epoch:16/40 Iteration:3820 | Train loss:0.00654
Epoch:16/40 Iteration:3840 | Train loss:0.00458
Epoch:16/40 Iteration:3860 | Train loss:0.00105
Epoch:16/40 Iteration:3880 | Train loss:0.00306
Epoch:16/40 Iteration:3900 | Train loss:0.00793
Epoch:16/40 Iteration:3920 | Train loss:

Epoch:28/40 Iteration:6940 | Train loss:0.00023
Epoch:28/40 Iteration:6960 | Train loss:0.00206
Epoch:28/40 Iteration:6980 | Train loss:0.04161
Epoch:28/40 Iteration:7000 | Train loss:0.00074
Epoch:29/40 Iteration:7020 | Train loss:0.00157
Epoch:29/40 Iteration:7040 | Train loss:0.00026
Epoch:29/40 Iteration:7060 | Train loss:0.00014
Epoch:29/40 Iteration:7080 | Train loss:0.00035
Epoch:29/40 Iteration:7100 | Train loss:0.00129
Epoch:29/40 Iteration:7120 | Train loss:0.00005
Epoch:29/40 Iteration:7140 | Train loss:0.00034
Epoch:29/40 Iteration:7160 | Train loss:0.00033
Epoch:29/40 Iteration:7180 | Train loss:0.00012
Epoch:29/40 Iteration:7200 | Train loss:0.00023
Epoch:29/40 Iteration:7220 | Train loss:0.00083
Epoch:29/40 Iteration:7240 | Train loss:0.00011
Epoch:30/40 Iteration:7260 | Train loss:0.00269
Epoch:30/40 Iteration:7280 | Train loss:0.00031
Epoch:30/40 Iteration:7300 | Train loss:0.00052
Epoch:30/40 Iteration:7320 | Train loss:0.00077
Epoch:30/40 Iteration:7340 | Train loss:

In [84]:
preds = rnn.predict(X_test)
y_true = y_test[:len(preds)]
print('test Acc.:{:.3f}%'.format(100 * (np.sum(preds == y_true) / len(y_true))))

INFO:tensorflow:Restoring parameters from models/sentiment-39.ckpt
test Acc.:50.000%


In [89]:
proba = rnn.predict(X_test, return_proba=True)

INFO:tensorflow:Restoring parameters from models/sentiment-39.ckpt


In [90]:
proba

array([1.0000000e+00, 8.9112341e-08, 9.9997389e-01, 9.9999952e-01,
       5.2278022e-09, 5.9258106e-08, 2.4045349e-07, 1.3626408e-09,
       6.0818791e-05, 4.6889502e-07, 4.7616226e-08, 2.7843949e-01,
       1.0000000e+00, 1.9271088e-08, 1.0030216e-03, 1.6595300e-04,
       1.0000000e+00, 9.9887389e-01, 7.2175670e-09, 1.0000000e+00,
       1.0000000e+00, 9.0367990e-08, 9.9999976e-01, 1.0000000e+00,
       5.0584714e-09, 9.3793562e-05, 1.0000000e+00, 1.2583264e-06,
       1.0000000e+00, 9.9643588e-01, 9.9994409e-01, 9.9999785e-01,
       1.0000000e+00, 1.0000000e+00, 9.9999976e-01, 1.0000000e+00,
       3.4491585e-08, 3.4103650e-03, 1.0000000e+00, 9.9535710e-01,
       1.0000000e+00, 1.0000000e+00, 3.6325947e-08, 9.9999976e-01,
       9.9999928e-01, 9.0872443e-10, 1.0000000e+00, 1.0000000e+00,
       1.9880648e-09, 9.9999905e-01, 4.9986163e-08, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0506316e-06, 9.9990702e-01,
       8.6225755e-09, 1.0000000e+00, 6.4979905e-01, 2.3436000e